1. 小区间质押是否可能获得高额利润

In [2]:
# 获取并存储天级数据
POOL_ID = "0xc31e54c7a869b9fcbecc14363cf510d1c41fa443"

arbitrum_query = """ query get_pools($num_skip: Int, $pool_id: ID!) {
    liquidityPool(id: $pool_id) {
        dailySnapshots(skip: $num_skip, orderBy: day, orderDirection: desc) {
            day
            dailyVolumeUSD
            dailyTotalRevenueUSD
            totalValueLockedUSD
        }
    }
}"""

In [ ]:
num_skip = 0
result = []
is_break = False

In [ ]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport


client = Client(
    transport=RequestsHTTPTransport(
        url='https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-arbitrum',
        verify=True,
        retries=5,
    ))

while True:
    print("Querying ticks, num_skip={}".format(num_skip))
    variables = {"num_skip": num_skip, "pool_id": POOL_ID}
    response = client.execute(gql(arbitrum_query), variable_values=variables)
    query_result = response["liquidityPool"]["dailySnapshots"]
    for item in query_result:
        day = int(item["day"])
        # todo: 修改这个部分
        if day <= 19477:
            is_break = True
            break
        result.append(item)

    if is_break == True or len(query_result) == 0:
        break

    num_skip = len(result)

In [4]:
import pandas as pd

from datetime import datetime, timedelta


def unixtime_to_day_before(day_count, unixtime=0):
    current_date = datetime.fromtimestamp(unixtime) + timedelta(days=day_count)
    return current_date.strftime("%Y-%m-%d")


arb_df = pd.DataFrame(result)
arb_df["date"] = arb_df["day"].apply(unixtime_to_day_before)
arb_df.head()

,day,dailyVolumeUSD,dailyTotalRevenueUSD,totalValueLockedUSD,date
0,19556,62645858.87339231038231371303131,31322.92943669615519115685651945,43991968.86397924299999433932886,2023-07-18
1,19555,78836539.34931248039460595756886,39418.2696746562401973029787891,44594966.108651452749034656584,2023-07-17
2,19554,43233649.01354901681799617409771,21616.82450677450840899808705174,46212787.3971710232147209161641,2023-07-16
3,19553,29838778.31630346908133923990871,14919.38915815173454066961995703,46504482.69780273868477842499168,2023-07-15
4,19552,129294121.16599334024223784584379,64647.06058299667012111892292883,46430790.59592811944603207193834,2023-07-14


In [6]:
arb_df["dailyVolumeUSD"] = arb_df["dailyVolumeUSD"].astype(float).round(2)
arb_df["dailyTotalRevenueUSD"] = arb_df["dailyTotalRevenueUSD"].astype(float).round(2)
arb_df["totalValueLockedUSD"] = arb_df["totalValueLockedUSD"].astype(float).round(2)
arb_df["date"] = arb_df["date"].astype(str)
arb_df = arb_df[['date', 'dailyVolumeUSD', 'dailyTotalRevenueUSD', 'totalValueLockedUSD']]

arb_old_df = pd.read_csv("../../data/arb_data_20230718.csv")
# arb_old_df.rename(columns={'dailyTotalVolumeUSD': 'dailyVolumeUSD'}, inplace=True)
arb_df = pd.concat([arb_df, arb_old_df])
arb_df.to_csv("../../data/arb_data_20230718.csv", index=False)

,date,dailyTotalVolumeUSD,dailyTotalRevenueUSD,totalValueLockedUSD
0,2023-04-30,1.294347e+08,64717.36,58969074.13
1,2023-04-29,5.831820e+07,29159.10,59522967.00
2,2023-04-28,1.291540e+08,64576.98,58623166.41
3,2023-04-27,2.670841e+08,133542.06,53907496.47
4,2023-04-26,2.884190e+08,144209.50,50551198.16


In [21]:
# 获取并存储小时级数据
POOL_ID = "0xc31e54c7a869b9fcbecc14363cf510d1c41fa443"

arbitrum_query = """ query get_pools($num_skip: Int, $pool_id: ID!) {
    liquidityPool(id: $pool_id) {
        hourlySnapshots(skip: $num_skip, orderBy: hour, orderDirection: desc) {
            hour
            hourlyVolumeUSD
            hourlyTotalRevenueUSD
            totalValueLockedUSD
        }
    }
}"""

num_skip = 0
result = []
is_break = False

In [24]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport


client = Client(
    transport=RequestsHTTPTransport(
        url='https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-arbitrum',
        verify=True,
        retries=5,
    ))

while True:
    print("Querying ticks, num_skip={}".format(num_skip))
    variables = {"num_skip": num_skip, "pool_id": POOL_ID}
    response = client.execute(gql(arbitrum_query), variable_values=variables)
    query_result = response["liquidityPool"]["hourlySnapshots"]
    for item in query_result:
        hour = int(item["hour"])
        # todo: 修改这个部分
        if hour <= 467775:
            is_break = True
            break
        result.append(item)

    if is_break == True or len(query_result) == 0:
        break

    num_skip = len(result)

Querying ticks, num_skip=0
Querying ticks, num_skip=100
Querying ticks, num_skip=200
Querying ticks, num_skip=300
Querying ticks, num_skip=400
Querying ticks, num_skip=500
Querying ticks, num_skip=600
Querying ticks, num_skip=700
Querying ticks, num_skip=800
Querying ticks, num_skip=900
Querying ticks, num_skip=1000
Querying ticks, num_skip=1100
Querying ticks, num_skip=1200
Querying ticks, num_skip=1300
Querying ticks, num_skip=1400
Querying ticks, num_skip=1500
Querying ticks, num_skip=1600


In [27]:
import time

def hour_to_unixtime(hour):
    current_date = datetime.fromtimestamp(0) + timedelta(hours=hour)
    return int(time.mktime(current_date.timetuple()))

In [29]:
arb_hour_df = pd.DataFrame(result)
arb_hour_df["timestamp"] = arb_hour_df["hour"].apply(hour_to_unixtime)
arb_hour_df.head()

,hour,hourlyVolumeUSD,hourlyTotalRevenueUSD,totalValueLockedUSD,timestamp
0,469376,1460878.47258988336766531052831,730.43923629494168383265526443,44233373.3510221773573453678896,1689753600
1,469375,806643.10919596517901233525319,403.3215545979825895061676268,44290156.20582897883020136578084,1689750000
2,469374,1036466.04480128782220493231525,518.23302240064391110246615779,44254886.55727854726053867876903,1689746400
3,469373,656345.72649494087873370659467,328.17286324747043936685329741,44168146.44106156636941494461425,1689742800
4,469372,1637624.49925289785898467064351,818.8122496264489294923353219,44087659.78889045440728093729744,1689739200


In [31]:
arb_hour_df[['timestamp', 'hourlyVolumeUSD', 'hourlyTotalRevenueUSD', 'totalValueLockedUSD']].to_csv("../../data/arb_hour_data_20230719.csv", index=False)

In [13]:
# 比较小时级数据回测质押收益 => 先粗略计算，再根据质押的分布精确计算

import pandas as pd

price_df = pd.read_csv("../../data/eth_usd_polygon_20230718.csv")
price_df.head()

,v,vw,o,c,h,l,t,n
0,9809.27,1942.71,1940.49,1952.54,1958.70,1928.00,1654041600000,30746
1,6089.57,1946.36,1954.20,1946.57,1954.20,1940.00,1654045200000,23571
2,8278.78,1935.83,1946.46,1939.03,1947.35,1928.98,1654048800000,25013
3,7490.52,1931.06,1938.94,1928.44,1940.24,1925.36,1654052400000,21634
4,10282.47,1924.81,1928.38,1932.54,1935.90,1912.03,1654056000000,25834


In [14]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

arb_df = pd.read_csv("../../data/arb_hour_data_20230719.csv")
arb_df.head()

,timestamp,hourlyVolumeUSD,hourlyTotalRevenueUSD,totalValueLockedUSD
0,1689753600,1460878.47,730.44,44233373.35
1,1689750000,806643.11,403.32,44290156.21
2,1689746400,1036466.04,518.23,44254886.56
3,1689742800,656345.73,328.17,44168146.44
4,1689739200,1637624.50,818.81,44087659.79


In [15]:
price_df = price_df[['o', 'c', 'h', 'l', 't']]
price_df["timestamp"] = (price_df['t'] / 1000).astype(int)
price_df.head()

,o,c,h,l,t,timestamp
0,1940.49,1952.54,1958.70,1928.00,1654041600000,1654041600
1,1954.20,1946.57,1954.20,1940.00,1654045200000,1654045200
2,1946.46,1939.03,1947.35,1928.98,1654048800000,1654048800
3,1938.94,1928.44,1940.24,1925.36,1654052400000,1654052400
4,1928.38,1932.54,1935.90,1912.03,1654056000000,1654056000


In [16]:
df = pd.merge(arb_df, price_df, on='timestamp')
df.head()

,timestamp,hourlyVolumeUSD,hourlyTotalRevenueUSD,totalValueLockedUSD,o,c,h,l,t
0,1689721200,1839294.64,919.65,43991968.86,1901.64,1897.72,1903.60,1893.29,1689721200000
1,1689717600,1570102.68,785.05,43971689.85,1897.51,1901.65,1904.20,1896.13,1689717600000
2,1689714000,1699318.65,849.66,44024583.09,1894.62,1897.49,1900.70,1892.85,1689714000000
3,1689710400,2210259.03,1105.13,43916277.35,1892.56,1894.61,1899.80,1889.99,1689710400000
4,1689706800,2770797.59,1385.40,43830655.08,1901.68,1892.67,1905.20,1890.43,1689706800000


In [17]:
df.shape

(1592, 9)

In [18]:
# get all timestamp in arb_df but not in df
from utils import utils

rest_df = arb_df[~arb_df['timestamp'].isin(df['timestamp'])]
rest_df["datetime"] = rest_df["timestamp"].apply(utils.utc_timestamp_to_datetime_str)
rest_df.head(10)

/var/folders/sj/9n8j6m2s6_1g9x78vzd8z0rm0000gn/T/ipykernel_1928/1548438947.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rest_df["datetime"] = rest_df["timestamp"].apply(utils.utc_timestamp_to_datetime_str)


,timestamp,hourlyVolumeUSD,hourlyTotalRevenueUSD,totalValueLockedUSD,datetime
0,1689753600,1460878.47,730.44,44233373.35,2023-07-19 08:00:00
1,1689750000,806643.11,403.32,44290156.21,2023-07-19 07:00:00
2,1689746400,1036466.04,518.23,44254886.56,2023-07-19 06:00:00
3,1689742800,656345.73,328.17,44168146.44,2023-07-19 05:00:00
4,1689739200,1637624.50,818.81,44087659.79,2023-07-19 04:00:00


In [23]:
from datetime import datetime


def timestamp_to_current_tz_datetime(ts):
    utc_date = datetime.fromtimestamp(ts)
    return utc_date.strftime("%Y-%m-%d %H:%M:%S")


df["datetime"] = df["timestamp"].apply(timestamp_to_current_tz_datetime)
df.tail()

,timestamp,hourlyVolumeUSD,hourlyTotalRevenueUSD,totalValueLockedUSD,o,c,h,l,t,datetime
1587,1684008000,2662455.28,1331.23,61592990.58,1793.70,1803.99,1805.90,1793.31,1684008000000,2023-05-14 04:00:00
1588,1684004400,5116826.36,2558.41,61377445.74,1790.12,1793.71,1801.65,1788.18,1684004400000,2023-05-14 03:00:00
1589,1684000800,6298006.55,3149.00,61259722.44,1793.99,1790.22,1808.40,1786.77,1684000800000,2023-05-14 02:00:00
1590,1683997200,2144822.87,1072.41,61175729.32,1794.92,1794.18,1799.50,1791.08,1683997200000,2023-05-14 01:00:00
1591,1683993600,2045304.59,1022.65,61190880.56,1796.19,1794.98,1797.30,1791.70,1683993600000,2023-05-14 00:00:00


In [24]:
df = df[["datetime", "h", "l", "hourlyTotalRevenueUSD", "totalValueLockedUSD"]]
df["interval"] = df["h"] - df["l"]
df.sort_values(by=["datetime"], inplace=True)
df.head()

,datetime,h,l,hourlyTotalRevenueUSD,totalValueLockedUSD,interval
1591,2023-05-14 00:00:00,1797.30,1791.70,1022.65,61190880.56,5.60
1590,2023-05-14 01:00:00,1799.50,1791.08,1072.41,61175729.32,8.42
1589,2023-05-14 02:00:00,1808.40,1786.77,3149.00,61259722.44,21.63
1588,2023-05-14 03:00:00,1801.65,1788.18,2558.41,61377445.74,13.47
1587,2023-05-14 04:00:00,1805.90,1793.31,1331.23,61592990.58,12.59


In [25]:
df.describe()

,h,l,hourlyTotalRevenueUSD,totalValueLockedUSD,interval
count,1592.00,1592.00,1592.00,1592.00,1592.00
mean,1852.10,1840.77,1833.37,52566694.77,11.33
std,69.83,69.73,2092.42,8176011.53,8.23
min,1641.40,1620.06,63.42,41703852.56,2.01
25%,1815.57,1806.49,658.20,44472744.53,6.46
50%,1863.85,1852.99,1211.43,48822178.50,8.85
75%,1901.60,1888.38,2132.02,60612443.02,13.28
max,2031.40,2011.75,26542.72,67373915.40,100.30


In [28]:
liquidity_df = pd.read_csv("../../data/arb_pool_liquidity.csv")
records = liquidity_df.to_dict("records")
print(records[0])

{'datetime': '2023-05-14 00:00:56', 'bottom_price': 1600.43980475, 'top_price': 1602.04096494, 'eth_amount': 0.0, 'usdc_amount': 105267.1299}


In [ ]:
def get_interval_liquidity(low_price, high_price):
    eth_amount = 0
    usdc_amount = 0
    for record in records:
        if low_price < record["bottom_price"] < high_price or low_price < record["top_price"] < high_price:
            eth_amount += record["eth_amount"]
            usdc_amount += record["usdc_amount"]
    return eth_amount, usdc_amount


hourly_data = df.to_dict("records")
